In [1]:
# import libs
import cv2
import mediapipe as mp
import time

from skimage.draw import line

In [5]:
mp_pose = mp.solutions.pose             # pose estimation utils

hm_ls = [0] * 33
prev_coord_ls = [[0,0] for _ in range(33)]

In [15]:
def draw_hm_skelton(img, lm, hm_ls, coord_ls, y, x, line_size):
    for i in range(33):
        hm_ls[i] = min(1, (abs(coord_ls[i][0] - lm[i].x) + abs(coord_ls[i][1] - lm[i].y)) * 10)
        coord_ls[i] = [lm[i].x, lm[i].y]

    # for item in mp_pose.POSE_CONNECTIONS:
    #     img = cv2.line(img, (int(lm[item[0]].x * x), int(lm[item[0]].y * y)),
    #                    (int(lm[item[1]].x * x), int(lm[item[1]].y * y)), (255,255,255), line_size)

    for item in mp_pose.POSE_CONNECTIONS:
        lines = list(zip(*line(int(lm[item[0]].x * x), int(lm[item[0]].y * y), int(lm[item[1]].x * x), int(lm[item[1]].y * y))))

        for i in range(0, int(len(lines) * hm_ls[item[0]])-1):
            img = cv2.line(img, (int(lines[i][0]), int(lines[i][1])), (int(lines[i+1][0]), int(
                lines[i+1][1])), (50 + int((1 - hm_ls[item[0]]) * 155) + i, 50 + int((1 - hm_ls[item[0]]) * 155) + i, 255 - int(hm_ls[0] * 50)), line_size)

        for i in range(len(lines) - 1, len(lines) - int(len(lines) * hm_ls[item[1]]), -1):
            img = cv2.line(img, (int(lines[i][0]), int(lines[i][1])), (int(
                lines[i-1][0]), int(lines[i-1][1])), (50 + int((1 - hm_ls[item[1]]) * 155) + i, 50 + int((1 - hm_ls[item[1]]) * 155) + i, 255 - int(hm_ls[1] * 50)), line_size)

    return img


In [ ]:

cap = cv2.VideoCapture('sample.mp4')    # capture

# # init output
# out = cv2.VideoWriter('output1.mp4', cv2.VideoWriter_fourcc(
#     *'mp4v'), 24, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))


with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        if ret:
            # Mediapipe required RGB format
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img.flags.writeable = False

            results = pose.process(img)  # get prediction

            img.flags.writeable = True
            # OpenCV uses BGR format
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            # try operations only if landmarks were found
            # try:
            lm = results.pose_landmarks.landmark    # landmarks
            # starttime = time.time()
            img = draw_hm_skelton(img, lm, hm_ls, prev_coord_ls, frame.shape[0], frame.shape[1], 40)
            # print(time.time() - starttime)
            # except:
                # pass

            # feed  
            cv2.imshow('feed', img)
            # out.write(img)

            # break feed
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        else:
            break

    # release feed
    cap.release()
    # out.release()
    cv2.destroyAllWindows()


In [ ]:
# 0.001-2-3